## Base Models

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
pos_final_test = pd.read_csv('./pos_final_test.csv.gz', compression='gzip')
pos_final_test_KNN = pd.read_csv('./pos_KNN_final_test.csv.gz', compression='gzip')
pos_final_missing_data_test = pd.read_csv('./pos_final_missing_data_test.csv.gz', compression='gzip')


In [3]:
model_cols = ~pos_final_test.columns.isin(['(\'subject_id\', \'\')', '(\'hadm_id\', \'\')', 'subject_id', 'hadm_id', 'Unnamed: 0', 'pos', 'ethnicity', 'gender', 'age'])

hadm_features = pos_final_test.loc[:, model_cols]
hadm_target = pos_final_test.loc[:, 'pos']

# hadm_features = pos_final_missing_data_test.loc[:, model_cols]
# hadm_target = pos_final_missing_data_test.loc[:, 'pos']


In [4]:
# # Creatinine - 50912
# hadm_features = hadm_features.loc[:, ['(\'max\', 50912)',  '(\'above_max\', 50912)', '(\'mean\', 50912)',  '(\'abn_percent\', 50912)']]

# # Least Missingness
# # [51221, 51265, 51222, 51301, 51279, 51277, 51248, 51249, 51250]
# hadm_features = hadm_features.loc[:, ['(\'max\', 51221)',  '(\'above_max\', 51221)', '(\'mean\', 51221)',  '(\'abn_percent\', 51221)',
#                                         '(\'max\', 51265)',  '(\'above_max\', 51265)', '(\'mean\', 51265)',  '(\'abn_percent\', 51265)',
#                                         '(\'max\', 51222)',  '(\'above_max\', 51222)', '(\'mean\', 51222)',  '(\'abn_percent\', 51222)',
#                                         '(\'max\', 51301)',  '(\'above_max\', 51301)', '(\'mean\', 51301)',  '(\'abn_percent\', 51301)',
#                                         '(\'max\', 51279)',  '(\'above_max\', 51279)', '(\'mean\', 51279)',  '(\'abn_percent\', 51279)',
#                                         '(\'max\', 51277)',  '(\'above_max\', 51277)', '(\'mean\', 51277)',  '(\'abn_percent\', 51277)',
#                                         '(\'max\', 51248)',  '(\'above_max\', 51248)', '(\'mean\', 51248)',  '(\'abn_percent\', 51248)',
#                                         '(\'max\', 51249)',  '(\'above_max\', 51249)', '(\'mean\', 51249)',  '(\'abn_percent\', 51249)',
#                                         '(\'max\', 51250)',  '(\'above_max\', 51250)', '(\'mean\', 51250)',  '(\'abn_percent\', 51250)']]

# # Least Missingness + Creatinine
# hadm_features = hadm_features.loc[:, ['(\'max\', 51221)',  '(\'above_max\', 51221)', '(\'mean\', 51221)',  '(\'abn_percent\', 51221)',
#                                         '(\'max\', 51265)',  '(\'above_max\', 51265)', '(\'mean\', 51265)',  '(\'abn_percent\', 51265)',
#                                         '(\'max\', 51222)',  '(\'above_max\', 51222)', '(\'mean\', 51222)',  '(\'abn_percent\', 51222)',
#                                         '(\'max\', 51301)',  '(\'above_max\', 51301)', '(\'mean\', 51301)',  '(\'abn_percent\', 51301)',
#                                         '(\'max\', 51279)',  '(\'above_max\', 51279)', '(\'mean\', 51279)',  '(\'abn_percent\', 51279)',
#                                         '(\'max\', 51277)',  '(\'above_max\', 51277)', '(\'mean\', 51277)',  '(\'abn_percent\', 51277)',
#                                         '(\'max\', 51248)',  '(\'above_max\', 51248)', '(\'mean\', 51248)',  '(\'abn_percent\', 51248)',
#                                         '(\'max\', 51249)',  '(\'above_max\', 51249)', '(\'mean\', 51249)',  '(\'abn_percent\', 51249)',
#                                         '(\'max\', 51250)',  '(\'above_max\', 51250)', '(\'mean\', 51250)',  '(\'abn_percent\', 51250)',
#                                      '(\'max\', 50912)',  '(\'above_max\', 50912)', '(\'mean\', 50912)',  '(\'abn_percent\', 50912)']]


In [5]:
X_train, X_test, y_train, y_test = train_test_split(hadm_features, hadm_target, stratify=hadm_target, random_state=0)


## Logistic Regression

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [7]:
%%time

# L2
params_log = [ {'C':np.logspace(-4, 3, 20)} ]
logReg = LogisticRegression(max_iter = 2000, class_weight = "balanced")

# # L1
# params_log = [ {'C':np.logspace(-6, -1, 20)} ]
# logReg = LogisticRegression(penalty = 'l1', max_iter = 2000, class_weight = "balanced", solver = 'liblinear')

grid_log = GridSearchCV(estimator = logReg, param_grid = params_log, cv = 5)
grid_log.fit(X_train, y_train)
grid_log.best_score_

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

CPU times: user 3min 50s, sys: 31.2 s, total: 4min 21s
Wall time: 44.8 s


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8257313046971893

In [8]:
grid_log.best_params_

{'C': 0.0012742749857031334}

In [19]:
est_coefs = pd.Series(grid_log.best_estimator_.coef_[0], index=X_train.columns)

sortedCoefs = np.abs(est_coefs).sort_values(ascending=False)
sortedCoefs[sortedCoefs > 0]

('abn_percent', 51006)    0.214087
('abn_percent', 51003)    0.199360
('min', 51006)            0.142296
('abn_percent', 50931)    0.140455
('mean', 51006)           0.134116
                            ...   
('above_max', 51265)      0.001165
('mean', 51506)           0.000692
('below_min', 50971)      0.000637
('mean', 51222)           0.000575
('min', 51221)            0.000411
Length: 225, dtype: float64

In [10]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

In [11]:
pred = grid_log.predict(X_test)

print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
print("_______________________________________________")
print("Classification Report:", end='')
print(f"\tPrecision Score: {precision_score(y_test, pred) * 100:.2f}%")
print(f"\t\t\tRecall Score: {recall_score(y_test, pred) * 100:.2f}%")
print(f"\t\t\tF1 score: {f1_score(y_test, pred) * 100:.2f}%")
print("_______________________________________________")
print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")

Accuracy Score: 81.37%
_______________________________________________
Classification Report:	Precision Score: 40.18%
			Recall Score: 73.00%
			F1 score: 51.83%
_______________________________________________
Confusion Matrix: 
 [[1559  326]
 [  81  219]]



## Random Forest

In [12]:
from sklearn import ensemble
import sklearn.model_selection as ms

randomForest = ensemble.RandomForestClassifier(class_weight = "balanced")

In [13]:
%%time

grid_para_forest = {
    'criterion': ['gini', 'entropy'],
    'max_depth': range(1, 20),
    'n_estimators': range(10, 110, 10)
}

grid_search_forest = ms.GridSearchCV(randomForest, grid_para_forest, scoring='accuracy', cv=5, n_jobs=-1)
grid_search_forest.fit(X_train, y_train)

CPU times: user 17.1 s, sys: 955 ms, total: 18 s
Wall time: 6min 46s


GridSearchCV(cv=5, estimator=RandomForestClassifier(class_weight='balanced'),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(1, 20),
                         'n_estimators': range(10, 110, 10)},
             scoring='accuracy')

In [14]:
# Get most important features
tree_final = grid_search_forest.best_estimator_
feature_importance = tree_final.feature_importances_
feature_importance = list(zip(X_train.columns, feature_importance))

feature_importance.sort(key = lambda x: x[1], reverse = True)
feature_importance

[("('max', 51006)", 0.04460647347298873),
 ("('above_max', 51006)", 0.0404782455086317),
 ("('mean', 51006)", 0.037441647839740286),
 ("('mean', 50912)", 0.02378874908711296),
 ("('abn_percent', 51006)", 0.023707749110205335),
 ("('max', 50912)", 0.01913332213644817),
 ("('mean', 51277)", 0.018135520932436002),
 ("('min', 51006)", 0.015424283957138862),
 ("('mean', 51237)", 0.014165955729248766),
 ("('max', 51237)", 0.014110421702973615),
 ("('min', 51003)", 0.01397082153242851),
 ("('max', 51277)", 0.01352661873423671),
 ("('max', 51003)", 0.01295377873426016),
 ("('mean', 51249)", 0.012443442388680033),
 ("('mean', 51003)", 0.012377652818029198),
 ("('min', 51249)", 0.012180996115830732),
 ("('min', 51277)", 0.011820488729795273),
 ("('above_max', 51274)", 0.01163573130291575),
 ("('mean', 51275)", 0.010658848917106148),
 ("('min', 51274)", 0.009970550020056246),
 ("('mean', 51274)", 0.00961495244398068),
 ("('min', 50912)", 0.009563256368114923),
 ("('max', 51274)", 0.00907669240182

In [15]:
grid_search_forest.best_params_

{'criterion': 'entropy', 'max_depth': 13, 'n_estimators': 90}

In [16]:
grid_search_forest.best_score_

0.8901278087352467

In [17]:
print("The training error is: %.5f" % (grid_search_forest.score(X_train, y_train)))
print("The test     error is: %.5f" % (grid_search_forest.score(X_test, y_test)))

The training error is: 0.99741
The test     error is: 0.88101


In [18]:
rf_predictions = tree_final.predict(X_test)

print(f"Accuracy Score: {accuracy_score(y_test, rf_predictions) * 100:.2f}%")
print("_______________________________________________")
print("Classification Report:", end='')
print(f"\tPrecision Score: {precision_score(y_test, rf_predictions) * 100:.2f}%")
print(f"\t\t\tRecall Score: {recall_score(y_test, rf_predictions) * 100:.2f}%")
print(f"\t\t\tF1 score: {f1_score(y_test, rf_predictions) * 100:.2f}%")
print("_______________________________________________")
print(f"Confusion Matrix: \n {confusion_matrix(y_test, rf_predictions)}\n")

Accuracy Score: 88.10%
_______________________________________________
Classification Report:	Precision Score: 61.76%
			Recall Score: 35.00%
			F1 score: 44.68%
_______________________________________________
Confusion Matrix: 
 [[1820   65]
 [ 195  105]]

